# Cython: A First Look

## Some Jupyter lab notes:
* Jupyter lab let's us make cells and run code in a nicely formatted way
* We also can use things like magic cells - these allow us to do special operations on code
* Rerunning cells is super easy
* For Cython - the notebook abstracts all of the compilation away
* Also for Cython - allows you to profile your code

## Typical sieve algorithm:

1. Create a list of integers 2 -> N
2. Start at 2, all factors of it are marked in the list as non-prime (false)
3. Go to next true index
4. Mark all factors of it in the list as false
5. Go to step 3
6. All remaining true indices are prime numbers

Here's a basic sieve implementation. Nothing special.

Might not even be the most efficient!

In [ ]:
def sieve(sieve_length):
    sieve_table = [True for x in range(sieve_length)]
    sieve_table[0] = False
    sieve_table[1] = False
    
    for i in range(2,int(sieve_length**0.5)+1):
        if sieve_table[i]:
            for marker in range(i*i, sieve_length, i):
                sieve_table[marker] = False
    
    return [i for i, t in enumerate(sieve_table) if t]

Testing base functionality:

In [ ]:
primes = sieve(1_000)
print(','.join([str(p) for p in primes]))

Everything appears to be working, but how fast is it?

Time for some basic benchmarking!

In [ ]:
%timeit sieve(1_000_000)

In [ ]:
%timeit sieve(10_000_000)

Anecdotally - I happen to know this is pretty slow.

## First steps into Cython

In [ ]:
%load_ext Cython

In [ ]:
%%cython

def sieve_magic(sieve_length):
    sieve_table = [True for x in range(sieve_length)]
    sieve_table[0] = False
    sieve_table[1] = False
    
    for i in range(2,int(sieve_length**0.5)+1):
        if sieve_table[i]:
            for marker in range(i*i, sieve_length, i):
                sieve_table[marker] = False
    
    return [i for i, t in enumerate(sieve_table) if t]

In [ ]:
primes_magic = sieve_magic(1_000)
print(','.join([str(p) for p in primes_magic]))

In [ ]:
%timeit sieve_magic(1_000_000)

# Exploring with Cython

Cython gives us the ability to view how our code has compiled!

Let's try it:

# Splitting things up

It looks like working on these list comprehensions is going to be a struggle... Let's split some things up.


# Calling STL Functions

At this point we know that there's more we can do with that inner for loop - but let's have a look at the list access that's being done.

Why don't we replace it with a C++ structure?

In [ ]:
%reload_ext Cython

In [ ]:
%%cython
# distutils: language=c++

import cython

from libcpp.vector cimport vector

def do_stuff():
    cdef vector[int] totally_a_list
    totally_a_list.push_back(100)
    return totally_a_list[0]

In [ ]:
do_stuff()

that was easy! Let's rewrite our previous code now.

# Battling the Inner Loop

There's other smaller optimizations to do for sure - but what about that inner for loop?